In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


In [3]:
df = pd.read_excel('MUNIC//Base_MUNIC_2020.xlsx', sheet_name='Agropecuário', usecols=['Cod UF','CodMun','Mun','Magr01','Magr03','Magr04','Magr05','Magr06'])
df

,CodMun,Cod UF,Mun,Magr01,Magr03,Magr04,Magr05,Magr06
0,1100015,11,Alta Floresta DOeste,Secretaria em conjunto com outras políticas se...,Masculino,42,Branca,Ensino superior completo
1,1100023,11,Ariquemes,Secretaria em conjunto com outras políticas se...,Feminino,42,Branca,Ensino superior completo
2,1100031,11,Cabixi,Secretaria exclusiva,Masculino,50,Parda,Ensino fundamental (1º Grau) incompleto
3,1100049,11,Cacoal,Secretaria em conjunto com outras políticas se...,Masculino,52,Branca,Ensino médio (2º Grau) completo
4,1100056,11,Cerejeiras,Secretaria em conjunto com outras políticas se...,Masculino,63,Branca,Ensino médio (2º Grau) completo
...,...,...,...,...,...,...,...,...
5565,5222005,52,Vianópolis,Secretaria exclusiva,Masculino,65,Parda,Ensino médio (2º Grau) completo
5566,5222054,52,Vicentinópolis,Secretaria em conjunto com outras políticas se...,Feminino,36,Parda,Ensino superior completo
5567,5222203,52,Vila Boa,Secretaria exclusiva,Masculino,52,Parda,Ensino médio (2º Grau) completo
5568,5222302,52,Vila Propício,Secretaria exclusiva,Masculino,52,Branca,Ensino médio (2º Grau) incompleto


In [4]:
cod_uf = pd.read_csv('MUNIC//MUNIC_educacao_2021.csv', sep=';')[['UF','Cod UF']]

In [5]:
x= cod_uf.pivot_table(columns=('UF','Cod UF'), aggfunc='size')


In [6]:
cod_uf = pd.DataFrame(x).reset_index()[['UF','Cod UF']]

In [7]:
df = df.merge(cod_uf, right_on='Cod UF',left_on='Cod UF')
df

,CodMun,Cod UF,Mun,Magr01,Magr03,Magr04,Magr05,Magr06,UF
0,1100015,11,Alta Floresta DOeste,Secretaria em conjunto com outras políticas se...,Masculino,42,Branca,Ensino superior completo,RO
1,1100023,11,Ariquemes,Secretaria em conjunto com outras políticas se...,Feminino,42,Branca,Ensino superior completo,RO
2,1100031,11,Cabixi,Secretaria exclusiva,Masculino,50,Parda,Ensino fundamental (1º Grau) incompleto,RO
3,1100049,11,Cacoal,Secretaria em conjunto com outras políticas se...,Masculino,52,Branca,Ensino médio (2º Grau) completo,RO
4,1100056,11,Cerejeiras,Secretaria em conjunto com outras políticas se...,Masculino,63,Branca,Ensino médio (2º Grau) completo,RO
...,...,...,...,...,...,...,...,...,...
5565,5222005,52,Vianópolis,Secretaria exclusiva,Masculino,65,Parda,Ensino médio (2º Grau) completo,GO
5566,5222054,52,Vicentinópolis,Secretaria em conjunto com outras políticas se...,Feminino,36,Parda,Ensino superior completo,GO
5567,5222203,52,Vila Boa,Secretaria exclusiva,Masculino,52,Parda,Ensino médio (2º Grau) completo,GO
5568,5222302,52,Vila Propício,Secretaria exclusiva,Masculino,52,Branca,Ensino médio (2º Grau) incompleto,GO


In [8]:
df['ano']=2020

In [9]:
df = df.rename(columns={'UF':'sigla_uf',
                        'Cod UF':'cod_uf',
                        'CodMun':'id_municipio',
                        'Mun':'nome_municipio',
                        'Magr01':'caracterizacao_orgao_gestor',
                        'Magr03':'genero',
                        'Magr04':'idade',
                        'Magr05':'cor_raca',
                        'Magr06':'grau_instrucao'})

In [10]:
df['caracterizacao_orgao_gestor']=df['caracterizacao_orgao_gestor'].str.title()

In [11]:
#padronizando os dados da coluna 'tipo_orgao'
df['caracterizacao_orgao_gestor']=np.where(df['caracterizacao_orgao_gestor']=='Não Informou','Sem dados',df['caracterizacao_orgao_gestor'])

#padronizando os dados da coluna 'genero'
df['genero']=np.where(df['genero']=='Não informou','Sem dados',df['genero'])
df['genero']=np.where(df['genero']=='-','Sem dados',df['genero'])

#padronizando os dados da coluna 'idade'
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='-',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Sem titular',np.nan,df['idade']) 


#padronizando os dados da coluna 'cor_raca'

df['cor_raca']=np.where(df['cor_raca']=='Não informou','Sem dados',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='-','Sem dados',df['cor_raca'])


#padronizando os dados da coluna 'escolaridade'
df['grau_instrucao']=np.where(df['grau_instrucao']=='-','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Não informou','Sem dados',df['grau_instrucao'])


In [12]:
df['idade'] =pd.to_numeric(df['idade']) #transformando em numerico 


In [13]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [14]:
df.columns

Index(['id_municipio', 'cod_uf', 'nome_municipio',
       'caracterizacao_orgao_gestor', 'genero', 'idade', 'cor_raca',
       'grau_instrucao', 'sigla_uf', 'ano', 'faixa_etaria'],
      dtype='object')

In [15]:
df=df[['ano', 'sigla_uf','nome_municipio','id_municipio','caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao', 'faixa_etaria']] #reorganizando as colunas

In [16]:
# criando dicionário
dict_esco = {'Ensino fundamental (1º Grau) incompleto':'Até Ensino Fundamental',
             'Ensino fundamental (1º Grau) completo':'Até Ensino Fundamental',
            'Ensino médio (2º Grau) incompleto':'Até Ensino Médio',
             'Ensino médio (2º Grau) completo':'Até Ensino Médio',
             'Ensino superior incompleto':'Até Ensino Superior Completo',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}

In [17]:
df = df.replace({'grau_instrucao':dict_esco}) #substituindo valores para padronizar

In [22]:
df.columns

Index(['ano', 'sigla_uf', 'nome_municipio', 'id_municipio',
       'caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao',
       'faixa_etaria'],
      dtype='object')

Subindo para o GBQ

In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          5570 non-null   int64   
 1   sigla_uf                     5570 non-null   object  
 2   nome_municipio               5570 non-null   object  
 3   id_municipio                 5570 non-null   int64   
 4   caracterizacao_orgao_gestor  5570 non-null   object  
 5   genero                       5570 non-null   object  
 6   cor_raca                     5570 non-null   object  
 7   grau_instrucao               5570 non-null   object  
 8   faixa_etaria                 4980 non-null   category
dtypes: category(1), int64(2), object(6)
memory usage: 397.3+ KB


In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação referente municipio'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Código do IBGE do município'),
        bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Caracterização do órgão no qual o gestor está'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação')
        ]


In [ ]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_agropecuaria_tipo_orgao')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=09d2a5b0-1342-4ffd-92c8-591346020d2d>